## ALL EXTRACT TIL SILVER I DENNE FILEN

In [55]:
import pandas as pd
import ast



In [56]:
filepath_in = 'bronze/'
filepath_out = 'silver/'

In [57]:
filename_in = 'distance.csv'
df = pd.read_csv(filepath_in+filename_in)

df["destination_addresses"]

for elements in df['destination_addresses']:
    values = elements.replace("'", "").replace("[", "").replace("]", "").split(',')
    destination_city, destination_country = values
    print("City:", destination_city)
    print("Country:", destination_country)


ilist = ast.literal_eval(df["rows"][0])

type(ilist)

City: Rhodes
Country:  Greece


list

In [58]:
origin = ast.literal_eval(df["origin_addresses"][0])

print(origin)

for i in origin:
    new_origin = i.split(',')
# origin_city, origin_destination = origin
origin_city, origin_country = new_origin

['Stavanger, Norway']


In [59]:
distance_text = ilist[0]['elements'][0]['distance']['text']
distance_value = ilist[0]['elements'][0]['distance']['value']

duration_text = ilist[0]['elements'][0]['duration']['text']
duration_value = ilist[0]['elements'][0]['duration']['value']


In [60]:
def calc_CO2(distance, plane_dict):
    liter_ton_register = plane_dict["liter_fuel_consumption_hour"]/2832
    speed_kmh = plane_dict["speed_kmh"]
    emission_factor = plane_dict["emission_factor"]
    
    answer = ((distance/speed_kmh) * liter_ton_register) * emission_factor
    return answer

In [61]:
# plane = {
#     "model":"Boeing 73788",
#     "liter_fuel_consumption_hour": 3200,
#     "speed_kmh": 875,
#     "emission_factor": 3.10
# }
# data["plane_model"] = plane["model"]
# distance = data["distance_value"][0]
# data["emissions"] = calc_CO2(distance, plane)

In [62]:
# Create a DataFrame
data = {
    'origin_city': [origin_city],
    'origin_country': [origin_country],
    'destination_city': [destination_city],
    'destination_country': [destination_country],
    'duration_text': [duration_text],
    'duration_value': [duration_value],
    'distance_text': [distance_text],
    'distance_value': [distance_value],
}


# Fictitious plane models
planes = [
    {"model": "Boeing 737-88", "liter_fuel_consumption_hour": 3200, "speed_kmh": 875, "emission_factor": 3.10},
    {"model": "Airbus A320", "liter_fuel_consumption_hour": 3100, "speed_kmh": 880, "emission_factor": 3.05},
    {"model": "Boeing 747", "liter_fuel_consumption_hour": 3500, "speed_kmh": 900, "emission_factor": 3.20},
    {"model": "Embraer E190", "liter_fuel_consumption_hour": 3000, "speed_kmh": 850, "emission_factor": 2.95},
    {"model": "Millennium Falcon", "liter_fuel_consumption_hour": 2500, "speed_kmh": 1050, "emission_factor": 2.50},
    {"model": "Starship Enterprise", "liter_fuel_consumption_hour": 2800, "speed_kmh": 920, "emission_factor": 2.80},
    {"model": "Hot Air Balloon", "liter_fuel_consumption_hour": 50, "speed_kmh": 20, "emission_factor": 0.10},
    {"model": "Fat Man On Unicycle", "liter_fuel_consumption_hour": 5, "speed_kmh": 1, "emission_factor": 0.05},
    {"model": "Thunderbird 2", "liter_fuel_consumption_hour": 2000, "speed_kmh": 800, "emission_factor": 2.00},
]
# Works for single plane


# Create a new DataFrame for emissions data
emissions_data = []

# Calculate emissions for each plane and append to the DataFrame
for plane in planes:
    distance = data["distance_value"][0]  # Use the same distance for all planes
    emissions = calc_CO2(distance, plane)
    emissions_data.append({"plane_model": plane["model"], "emissions": emissions})

df = pd.DataFrame(data)
emissions_df = pd.DataFrame(emissions_data)


# Duplicate rows in the base DataFrame based on the length of emissions DataFrame, to avoid null values, as there's nothing to join on
df_duplicate = pd.concat([df] * len(emissions_df), ignore_index=True)

# Concatenate the duplicated base DataFrame with emissions DataFrame
df_to_silver = pd.concat([df_duplicate, emissions_df], axis=1)

# Display the DataFrame
print(df_to_silver.to_json(orient='records', lines=True))

{"origin_city":"Stavanger","origin_country":" Norway","destination_city":"Rhodes","destination_country":" Greece","duration_text":"2 days 1 hour","duration_value":175461,"distance_text":"4,233 km","distance_value":4232706,"plane_model":"Boeing 737-88","emissions":16944.4889104116}
{"origin_city":"Stavanger","origin_country":" Norway","destination_city":"Rhodes","destination_country":" Greece","duration_text":"2 days 1 hour","duration_value":175461,"distance_text":"4,233 km","distance_value":4232706,"plane_model":"Airbus A320","emissions":16058.4534018683}
{"origin_city":"Stavanger","origin_country":" Norway","destination_city":"Rhodes","destination_country":" Greece","duration_text":"2 days 1 hour","duration_value":175461,"distance_text":"4,233 km","distance_value":4232706,"plane_model":"Boeing 747","emissions":18599.461393597}
{"origin_city":"Stavanger","origin_country":" Norway","destination_city":"Rhodes","destination_country":" Greece","duration_text":"2 days 1 hour","duration_valu

In [63]:
filename_out = "distance.json"
df_to_silver.to_json(filepath_out+filename_out,orient='records', lines=False)